In [6]:
import re
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import zipfile
from nlp.preprocessing import (
    clean_text,
    preprocess,
    tokenize,
    preprocess_document,
    tokenize_document,
    get_stopwords,
    lemmatization)
from nlp.text_statistics import (
    number_tokens,
    tokens_length,
    unique_tokens,
    count_numbers,
    number_stopwords,
    print_statistics)
from gensim.parsing.preprocessing import (
    strip_multiple_whitespaces,
    strip_non_alphanum,
    strip_punctuation2,
    strip_short)
from textpp_ptbr.preprocessing import TextPreProcessing as tpp

In [ ]:
file = './dados/itens_pregao_pitem_saude.csv'
data = pd.read_csv(file, sep=';')

data.info()
items = list(data['nom_item'])

In [ ]:
items_descriptions = preprocess(items, True, False)

In [9]:
dictionary_file = '../dados/dicionario/delaf.dic.zip'

In [ ]:
canonical_forms = collections.defaultdict(list)
word_class = {}

with zipfile.ZipFile(dictionary_file, 'r') as zipped:
    with zipped.open('delaf.dic', 'r') as data:
        lines = data.readlines()
        for line in lines:
            word_canonical = re.split(r'[,.+:\s]\s*', line)
            word = tpp.remove_accents(word_canonical[0])
            canonical = tpp.remove_accents(word_canonical[1])
            wclass = word_canonical[2]
            canonical_forms[word].append(canonical)
            if canonical in word_class and wclass == 'N':
                word_class[canonical] = wclass
            elif canonical not in word_class:
                word_class[canonical] = wclass

In [ ]:
canonical_form = {}

for word, canonical in canonical_forms.items():
    if len(canonical) == 1:
        canonical_form[word] = canonical[0]
    else:
        noun = ''
        for c in canonical:
            if word == 'milho':
                print(c)
            if word_class[c] == 'N':
                noun = c
                break
        if noun == '':
            canonical_form[word] = canonical[0]
        else:
            canonical_form[word] = noun

In [ ]:
len(canonical_form)

In [ ]:
descriptions_lemmatized = []
changes = 0
i = 0

for doc in items_descriptions:
    tokens = []
    for tok in doc:
        if tok in canonical_form:
            tokens.append(canonical_form[tok])
            if tok != canonical_form[tok]:
                changes += 1
        else:
            tokens.append(tok)
    if i < 1000:
        i += 1
        print(doc)
        print(tokens)
    descriptions_lemmatized.append(tokens)

In [ ]:
len(descriptions_lemmatized)